<a href="https://colab.research.google.com/github/vikramkalabi/Swift4TF-Playground/blob/master/SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import TensorFlow

In [0]:
public struct Fire : Layer {
    
    var convSqueeze: Conv2D<Float>
    var conv1Expand: Conv2D<Float>
    var conv3Expand: Conv2D<Float>
    
    public init (in_channels:Int, squeeze: Int, expand: Int) {
        
        convSqueeze = Conv2D<Float>(
            filterShape: (1, 1, in_channels, squeeze),
            activation: relu
        )
        conv1Expand = Conv2D<Float>(
            filterShape: (1, 1, squeeze, expand),
            activation: relu
        )
        conv3Expand = Conv2D<Float>(
            filterShape: (3, 3, squeeze, expand), 
            padding: .same,
            activation: relu
        )
        
    }
    
    @differentiable
    public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        let squeezed = convSqueeze(input)
        let expand1 = conv1Expand(squeezed)
        let expand3 = conv3Expand(squeezed)

        return expand1.concatenated(with: expand3, alongAxis: -1)
    }
}

In [0]:
func testFireModule(){
    let x = Tensor<Float>(randomUniform: [10, 28, 28, 32])
    let fire = Fire(in_channels: 32, squeeze: 16, expand: 64)
    let out  = try! fire(x)
    assert(out.shape == [10, 28, 28, 128])
}

testFireModule()

In [0]:
public struct SqueezeNet: Layer {
    
    // 32x32x3 -> 15x15x64
    var conv1 = Conv2D<Float>(
        filterShape: (3, 3, 3, 64), 
        padding: .valid,
        activation: relu 
    )
    // 15x15x64 -> 15x15x128
    var fire1 = Fire(in_channels: 64, squeeze: 16, expand: 64)
    // 15x15x128 -> 15x15x128
    var fire2 = Fire(in_channels: 64*2, squeeze: 16, expand: 64)
    // 15x15x128 -> 7x7x128
    var maxPool = MaxPool2D<Float>(poolSize:(3, 3), strides: (2, 2))
    // 7x7x128 ->  7x7x256
    var fire3 = Fire(in_channels: 64*2, squeeze:32, expand: 128)
    // 7x7x256 ->  7x7x256
    var fire4 = Fire(in_channels: 128*2, squeeze: 32, expand: 128)
    var drop = Dropout<Float>(probability: 0.5)
    // 7x7x256 ->  7x7xnumClasses
    var reduce: Conv2D<Float> 
    // 7x7xnumClasses ->  numClasses
    var globalPool = GlobalAvgPool2D<Float>()
    
    public init(_ numClasses: Int){
        reduce = Conv2D<Float>(
            filterShape: (7, 7, 256, numClasses),
            activation: relu
        )
    } 
    
    @differentiable
    public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        let convolved = conv1(input)
        let fired1 = convolved.sequenced(through: fire1, fire2, maxPool)
        let fired2 = fired1.sequenced(through: fire3, fire4)
        let prediction = fired2.sequenced(through: drop, reduce, globalPool)
        return prediction
    }
    
    

}

In [0]:
func testSqueezeNet(){
    let x = Tensor<Float>(randomUniform: [10, 32, 32, 3])
    let model = SqueezeNet(10)
    let out  = try! model(x)
    assert(out.shape == [10, 10], "\(out.shape)")
}

testSqueezeNet()